In [1]:
from utili.misc.misc import Adjust_Accelerator_settings,Create_Dataset_Result_Folder
import dill
import torch
import os

In [ ]:
import Dataset.FLAIR as Dataset
DATrain=Dataset.DATrain
cfg=Dataset.cfg

In [ ]:
#Interface #
####################################Parameters############################################################


Model_list=[]
GPU_NUM=0
Exp_name='Dummy'
models_initilization_seed_list=[41,50,61,70,81,90,101,200,301,400]
LR=cfg['LR']
EPOCH =cfg['EPOCH']
Num_of_Samples=cfg['Num_of_Samples']

############################################################################################################################

devices_DA, num_workers_DA, accelerator_type_DA= Adjust_Accelerator_settings(GPU_NUM=GPU_NUM)[3:]
for i in range(len(models_initilization_seed_list[:])):
    Paths =Create_Dataset_Result_Folder(Dataset,Exp_name)
    models_initilization_seed = models_initilization_seed_list[i]
    temp,DA_Datamodule,Light_source_trainer,best_model_path=DATrain(Dataset,batch_size=1,EPOCH=EPOCH,num_workers=num_workers_DA,Result_path=Paths[-2],random_seed=models_initilization_seed_list[i],lightgin_log_path=Paths[1],keep_train=False,Exp_name=Exp_name,LR=LR,devices=devices_DA,accelerator=accelerator_type_DA,Num_of_Samples=Num_of_Samples)
    ckp=torch.load(best_model_path, map_location='cpu')
    state_dict = ckp['state_dict']
    adjusted_state_dict = {k.replace('source_model.', ''): v for k, v in state_dict.items()}
    temp.load_state_dict(adjusted_state_dict, strict=True)
    S_cnn=temp
    Model_list.append(S_cnn)
    with open(os.path.join(Paths[0],Dataset.Dataset_name+"_lists_4.pkl"), 'wb') as f:
        dill.dump( Model_list, f)   